# Smart Cart Analytics. Part 2

## Sprint 2: Project Initiation

# INDEX

[Data Exploration. Part 2.Part2](#EDA)

[TRAIN DATASET](#1)

[PRIOR DATASET](#2)

<a id='INDEX'></a>

## Data Exploration. Part 2

Continue read in the data and performing any required data cleaning steps. Combine the tables, remove redundant columns,  and 

<a id='EDA'></a>

In [7]:
 # importing required libraries:
import pandas as pd
import numpy as np

In [8]:
df_aisles = pd.read_csv('data/aisles.csv')
df_departments = pd.read_csv('data/departments.csv')
df_products = pd.read_csv('data/products.csv')
df_products_aisles = pd.merge(df_products, df_aisles, on='aisle_id')
df_products_final = pd.merge(df_products_aisles, df_departments, on='department_id')

#### Let´s have an overview at **Order products train** and **Order products  prior** data sets:

In [9]:
df_products_train = pd.read_csv('data/order_products__train.csv',index_col=0)
df_products_prior = pd.read_csv('data/order_products__prior.csv',index_col=0)

In [10]:
df_orders = pd.read_csv('data/orders.csv')

In [11]:
df_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


The data includes observations related to orders realized by users with the **ID information** of the user and their **order**, **Day of the week** when the order was realized, **hour of day** and the number of **days since prior order**. No outliers or wrong data stand out above as it is a clean data set.

What categorical data is stored: How is it distributed:

In [12]:
df_orders["eval_set"].value_counts()

eval_set
prior    3214874
train     131209
test       75000
Name: count, dtype: int64

The data set contains numerical and categorical data. Column **"eval_set"** contains prior, train and test data.

The **prior set** is used for feature engineering and understanding historical customer behavior.

The **train set** is used to train machine learning models to predict future behavior based on historical data.

The **test set** is used to evaluate the performance of the models in predicting unseen data. The actual labels for this set are usually withheld in a competition setting to test the model's predictive power. 

The NaN value in the column **days_since_prior_order** suggests it's the user's first order or the data for the prior order is not available.

The only column with the null (NaN) values is *¨Days since prior order¨*. These are the users that did not have prior orders on the platform. Lets see the percentage of the null values:

In [13]:
df_orders.isna().mean()*100

order_id                  0.000000
user_id                   0.000000
eval_set                  0.000000
order_number              0.000000
order_dow                 0.000000
order_hour_of_day         0.000000
days_since_prior_order    6.027594
dtype: float64

There are 6% of new users in the data set.

In [14]:
# Lets replace null values for 0 days.
df_orders_final = df_orders.fillna({'days_since_prior_order': 0}, inplace=False)

In [15]:
# Remove the columns we wont use for the analysis - 'day of the week', 'hour of day':
df_orders_final = df_orders_final.drop(['order_dow', 'order_hour_of_day'], axis=1)
df_orders_final

,order_id,user_id,eval_set,order_number,days_since_prior_order
0,2539329,1,prior,1,0.0
1,2398795,1,prior,2,15.0
2,473747,1,prior,3,21.0
3,2254736,1,prior,4,29.0
4,431534,1,prior,5,28.0
...,...,...,...,...,...
3421078,2266710,206209,prior,10,29.0
3421079,1854736,206209,prior,11,30.0
3421080,626363,206209,prior,12,18.0
3421081,2977660,206209,prior,13,7.0


Lets create and save *df_orders_prior* and *df_orders_train* datasets:

In [16]:
df_orders_prior = df_orders_final[df_orders_final['eval_set'] == 'prior']
df_orders_train = df_orders_final[df_orders_final['eval_set'] == 'train']
df_orders_test = df_orders_final[df_orders_final['eval_set'] == 'test']

In [17]:
df_orders_prior = df_orders_prior.drop(columns=['eval_set'])
df_orders_train = df_orders_train.drop(columns=['eval_set'])
df_orders_test = df_orders_test.drop(columns=['eval_set'])

In [18]:
df_orders_prior.to_csv("data/df_orders_prior.csv", index=False, header=True)
df_orders_train.to_csv("data/df_orders_train.csv", index=False, header=True)

Now we have the dataset separated per eval_set value in three separate ones: prior, test and train.

Lets combine the order data sets with product data sets for both $Prior$ and $Train$ data: We are not provided product data sets for test data in order to avoid model data leakage.

### TRAIN DATASET:

<a id='1'></a>

In [19]:
df_products_train

,product_id,add_to_cart_order,reordered
order_id,,,
1,49302,1,1
1,11109,2,1
1,10246,3,0
1,49683,4,0
1,43633,5,1
...,...,...,...
3421063,14233,3,1
3421063,35548,4,1
3421070,35951,1,1


In [20]:
df_orders_train

,order_id,user_id,order_number,days_since_prior_order
10,1187899,1,11,14.0
25,1492625,2,15,30.0
49,2196797,5,5,6.0
74,525192,7,21,6.0
78,880375,8,4,10.0
...,...,...,...,...
3420838,2585586,206199,20,30.0
3420862,943915,206200,24,6.0
3420924,2371631,206203,6,30.0
3420933,1716008,206205,4,10.0


In [21]:
# Merge the DataFrames on the 'order_id' column for the Train data:
df_merge_train = pd.merge(df_products_train,df_orders_train, on='order_id')
df_merge_train

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order
0,1,49302,1,1,112108,4,9.0
1,1,11109,2,1,112108,4,9.0
2,1,10246,3,0,112108,4,9.0
3,1,49683,4,0,112108,4,9.0
4,1,43633,5,1,112108,4,9.0
...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,30,4.0
1384613,3421063,35548,4,1,169679,30,4.0
1384614,3421070,35951,1,1,139822,15,8.0
1384615,3421070,16953,2,1,139822,15,8.0


Now, we have all data combined (product ids, user ids, reorder data, order number and days since prior order) in this merged dataframe for train orders.

In [ ]:
# Save the dataframe:
df_merge_train.to_csv("data/df_merge_train.csv", index=False, header=True)

In [167]:
#Lets review our products information table to remember how it looks like:
df_products_final

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks
...,...,...,...,...,...,...
49683,22827,Organic Black Mission Figs,18,10,bulk dried fruits vegetables,bulk
49684,28655,Crystallized Ginger Chunks,18,10,bulk dried fruits vegetables,bulk
49685,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk
49686,38007,Naturally Sweet Plantain Chips,18,10,bulk dried fruits vegetables,bulk


In [168]:
# Adding the table with product information:
df_train = pd.merge(df_merge_train,df_products_final, on='product_id')

In [170]:
df_train

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,112108,4,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,47901,14,16.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,2993,15,7.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,41425,4,14.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,187205,5,30.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,177077,4,30.0,Sprinkles Decors,97,13,baking supplies decor,pantry
1384613,3420084,47935,20,0,9808,9,30.0,Classic Original Lip Balm SPF 12,73,11,facial care,personal care
1384614,3420084,9491,21,0,9808,9,30.0,Goats Milk & Chai Soap,25,11,soap,personal care
1384615,3420088,16380,12,0,72444,14,5.0,Stevia Sweetener,97,13,baking supplies decor,pantry


Finally, we have the final version of our train orders dataset with all information from the initial source: order id, product id, add to cart order, reordered status, iser id, order number, days since prior order, product name, aisle name and id, department name and id.

### PRIOR DATASET:

<a id='2'></a>

In [171]:
# Merge the DataFrames on the 'order_id' column for the Prior data:
df_merge_prior = pd.merge(df_products_prior,df_orders_prior, on='order_id')
df_merge_prior

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order
0,2,33120,1,1,202279,3,8.0
1,2,28985,2,1,202279,3,8.0
2,2,9327,3,0,202279,3,8.0
3,2,45918,4,1,202279,3,8.0
4,2,30035,5,0,202279,3,8.0
...,...,...,...,...,...,...,...
32434484,3421083,39678,6,1,25247,24,21.0
32434485,3421083,11352,7,0,25247,24,21.0
32434486,3421083,4600,8,0,25247,24,21.0
32434487,3421083,24852,9,1,25247,24,21.0


Now, we have all data combined (product ids, user ids, reorder data, order number and days since prior order) in this merged dataframe for prior orders.

In [172]:
df_merge_prior.to_csv("data/df_merge_prior.csv", index=False, header=True)

In [173]:
# Adding the table with product information:
df_prior = pd.merge(df_merge_prior,df_products_final, on='product_id')

In [174]:
df_prior

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,202279,3,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,26,33120,5,0,153404,2,7.0,Organic Egg Whites,86,16,eggs,dairy eggs
2,120,33120,13,0,23750,11,10.0,Organic Egg Whites,86,16,eggs,dairy eggs
3,327,33120,5,1,58707,21,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
4,390,33120,28,1,166654,48,9.0,Organic Egg Whites,86,16,eggs,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,3265099,43492,3,0,111311,2,5.0,Gourmet Burger Seasoning,104,13,spices seasonings,pantry
32434485,3361945,43492,19,0,142932,25,5.0,Gourmet Burger Seasoning,104,13,spices seasonings,pantry
32434486,3267201,33097,2,0,183867,19,3.0,Piquillo & Jalapeno Bruschetta,81,15,canned jarred vegetables,canned goods
32434487,3393151,38977,32,0,79056,1,0.0,Original Jerky,100,21,missing,missing


In [175]:
# Saving our dfs
df_prior.to_csv("data/df_prior.csv", index=False, header=True)

In [176]:
df_train.to_csv("data/df_train.csv", index=False, header=True)

We will continue conducting EDA in the next part focusing on order items and department/isles of these products. We will conduct EDA on a bigger dataset *prior* but will use *test* dataset for further modeling.

### [BACK TO INDEX](#INDEX)